In [40]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [38]:
# Extract job list from search result page
def parse(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
    for each in soup.find_all(class_= "result" ):
        try: 
            title = each.find(class_='jobtitle').text.replace('\n', '')
        except:
            title = 'None'
        try:
            location = each.find('span', {'class':"location" }).text.replace('\n', '')
        except:
            location = 'None'
        try: 
            company = each.find(class_='company').text.replace('\n', '')
        except:
            company = 'None'
        try:
            salary = each.find('span', {'class':'no-wrap'}).text
        except:
            salary = 'None'
        try:
            url = each.find('a').attrs['href']
            job_url = 'htpp://indeed.com' + url
        except:
            job_url = 'http://indeed.com'
        synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
        df = df.append({'Title':title, 'Location':location, 'Company':company, 'URL':job_url ,'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
    return df

In [39]:
parse('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10')

,Title,Location,Company,Salary,Synopsis,URL
0,Jr Data Scientist,"New York, NY",Callière Group,None,Experience using some of the major...,htpp://indeed.com/rc/clk?jk=0875c173cf80e8c3&f...
1,Data Scientist,"New York, NY 10001 (Chelsea area)",Comcast,None,Freewheel is currently looking to ...,htpp://indeed.com/rc/clk?jk=c25dd7bf92018138&f...
2,"Data Scientist, Analytics","New York, NY",Slack,None,You should have deep technical ski...,htpp://indeed.com/rc/clk?jk=3220345942a1625f&f...
3,Data Scientist Intern,"Armonk, NY 10504",IBM,None,You will work alongside consultant...,htpp://indeed.com/rc/clk?jk=f41b0ee76cebe5f9&f...
4,Junior Data Scientist,"New York, NY",Interactive Brokers,None,Interactive Brokers Group (IBG) is...,htpp://indeed.com/rc/clk?jk=1ff7ee9cdc7a8fec&f...
5,Data Analyst,"Manhattan, NY",HUMAN RIGHTS COMMISSION,"\n $70,286 - $82,244 a year","Advise, as needed, on data governa...",htpp://indeed.com/rc/clk?jk=ce1824189e846b1a&f...
6,Data Scientist,"Manhattan, NY",Equityzen,None,3+ years of experience working wit...,htpp://indeed.com/rc/clk?jk=e642ed1c1bbb7bb3&f...
7,Data Scientist,"New York, NY",Twitter,None,The Innovation team is made up of ...,htpp://indeed.com/rc/clk?jk=b89c86bec775ae43&f...
8,Junior Data Analyst,"New York, NY",compstak.com,None,Support and learn from our team of...,htpp://indeed.com/rc/clk?jk=97f36b381a02b201&f...
9,Data Analyst,"Manhattan, NY",NYC Administration for Children's Serv...,"\n $70,286 - $83,784 a year",Support additional ad-hoc data req...,htpp://indeed.com/rc/clk?jk=4e3bb49c1c217e9c&f...


In [58]:
def job_description_extraction(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    job = pd.DataFrame()
    job_title = soup.find('h3', attrs={'class':'jobsearch-JobInfoHeader-title'}).getText()
    company = soup.find('h4', attrs={'class':'jobsearch-CompanyReview--heading'}).getText()
    job_description = soup.find('div', attrs={'class':'jobsearch-JobComponent-description'}).getText()
    job = job.append({'job_title': job_title, 'company':company, 'job_description':job_description}, ignore_index=True)
    return job

In [59]:
job_description_extraction('https://www.indeed.com/viewjob?jk=eabfb767c41a478c&from=serp&vjs=3')

,company,job_description,job_title
0,Red Bull,Wings Team Member Helsinki\nStudent Jobs\nThe ...,Wings Team Member Helsinki


In [56]:
# Parse jobs in Helsinki - v1
for start in range(0, 50, 10):
    page = requests.get('https://www.indeed.fi/jobs?q=&l=helsinki&start='+str(start))
    time.sleep(0.1) #Ensuring at least 2 sec between page grabs
    soup = BeautifulSoup(page.content, 'html.parser', from_encoding="utf-8")
        
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
    for each in soup.find_all(class_= "result" ):
        try: 
            title = each.find(class_='jobtitle').text.replace('\n', '')
        except:
            title = 'None'
        try:
            location = each.find('span', {'class':"location" }).text.replace('\n', '')
        except:
            location = 'None'
        try: 
            company = each.find(class_='company').text.replace('\n', '')
        except:
            company = 'None'
        try:
            salary = each.find('span', {'class':'no-wrap'}).text
        except:
            salary = 'None'
        try:
            url = each.find('a').attrs['href']
            job_url = 'http://indeed.com' + url
        except:
            job_url = 'http://indeed.com'
        synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
        df = df.append({'Title':title, 'Location':location, 'Company':company, 'URL':job_url ,'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)

df.to_csv('indeed_helsinki_1.csv', encoding='utf-8')


In [55]:
# Parse jobs in Helsinki - v2

df = pd.DataFrame(columns=['URL', 'Title', 'Company', 'Location', 'Summary', 'Salary'])

for start in range(0, 50, 10):
    page = requests.get('https://www.indeed.fi/jobs?q=&l=helsinki&start='+str(start))
    time.sleep(1)  #ensuring at least 1 second between page grabs
    soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
        
    for div in soup.find_all(name="div", attrs={"class":"row"}):
            #creating an empty list to hold the data for each posting
        job_post = []
        num = (len(sample_df) + 1)
            
            
            #grabbing job title
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs_link = "%s%s" % ('http://indeed.com',div.find('a').attrs['href'])
            title = a["title"]
                #grabbing company name
            company = div.find_all(name='span', attrs={'class':'company'})
            if len(company) > 0:
                for b in company:
                    company = b.text.strip()
            else:
                sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
                for span in sec_try:
                    company = span.text
            
            
            #grabbing location name
        c = div.findAll("span", attrs={"class":"location"})
        for span in c:
            location = span.text
            
            
            #grabbing summary text
        d = div.findAll("span", attrs={"class":"summary"})
        for span in d:
            summary = span.text.strip()
                
                
            #grabbing salary
        try:
            salary = div.find("nobr").text
        except:
            try:
                div_two = div.find(name="div", attrs={"class":"sjcl"})
                div_three = div_two.find("div")
                salary = div_three.text.strip()
            except:
                salary = "Nothing_found"
         
        df = df.append({'Title':title, 'Location':location, 'Company':company, 'URL':jobs_link ,'Salary':salary, 'Summary':summary}, ignore_index=True)
        

#saving sample_df as a local csv file — define your own local path to save contents 
df.to_csv("indeed_helsinki_2.csv", encoding='utf-8')

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
